In [728]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import re

In [729]:
data = pd.read_csv('Kosalge.csv')
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [730]:
coaches = data['Name'].values.tolist()
coaches = [coach.replace('Jr.,','') for coach in coaches]
ids = data['CoachID'].values.tolist()

special_cases = {'Michael Jr.': 'Mick',
                 'Mark "Bo"': 'Bo',
                 'D.J.': 'DJ',
                 'Van Gorder': 'VanGorder'}

for i in range(len(coaches)):
    coach = coaches[i]
    for special_case in special_cases:
        if special_case in coach:
            #print('True')
            coaches[i] = coach.replace(special_case, special_cases[special_case])

#print(coaches)

coaches = [(coach.split(', ')[-1]+'_'+('_'.join(coach.split(' ')[:-1])))[:-1] for coach in coaches]
coaches = [coach.strip() for coach in coaches]
coaches = [coach.replace('.','._').replace('__','_') for coach in coaches]
for i in range(len(coaches)):
    coaches[i] = [ids[i], coaches[i]]
coaches

[[206280, 'Dave_Ragone'],
 [483160, 'Dan_Campbell'],
 [288004, 'Aaron_Glenn'],
 [610308, 'Brian_Johnson'],
 [174517, 'Luke_Getsy'],
 [995605, 'Shane_Bowen'],
 [373812, 'Joe_Brady'],
 [283478, 'Mick_Lombardi'],
 [370429, 'Renaldo_Hill'],
 [780072, 'Scott_Turner'],
 [882423, 'Brian_VanGorder'],
 [403379, 'Adam_Stenavich'],
 [425989, 'Jonathan_Gannon'],
 [958787, 'Tim_Kelly'],
 [726241, 'Ronnie_Jones'],
 [285329, 'Kellen_Moore'],
 [737211, 'John_Butler'],
 [110664, 'Jerome_Henderson'],
 [992998, 'Andrew_Weidinger'],
 [472990, 'Buddy_Morris'],
 [539629, 'Nick_Rapone'],
 [375072, 'Jay_Rodgers'],
 [138235, 'Randy_Brown'],
 [472657, 'Rock_Gullickson'],
 [756239, 'Jake_Peetz'],
 [527587, 'Luke_Richesson'],
 [851071, 'Ronald_Curry'],
 [667492, 'Tom_Kanavy'],
 [295643, 'Chip_Morton'],
 [666583, 'Mike_Mallory'],
 [431842, 'Marcus_Robertson'],
 [847526, 'Jerry_Palmieri'],
 [369945, 'Joe_Gilbert'],
 [993089, 'Mike_Woicik'],
 [406496, 'Tom_Myslinski'],
 [105087, 'Bob_Rogucki'],
 [301889, 'Robert_Leo

In [731]:
valid_counter = 0
URLs = []
suffixes = ['_(American_football_coach)', '_(American_football)']
for coach in coaches:
    URL = 'https://en.wikipedia.org/wiki/'+coach[1]
    if requests.get(URL).status_code == 200:
        valid_counter += 1
        URLs.append([coach[0], URL])
    else:
        for suffix in suffixes:
            if requests.get(URL+suffix).status_code == 200:
                valid_counter += 1
                URLs.append([coach[0], URL+suffix])
                continue

print('Number of Valid URLs: ' + str(valid_counter))
print('% of Valid URLs: ' + str(valid_counter/len(coaches)))

Number of Valid URLs: 197
% of Valid URLs: 0.7035714285714286


In [732]:
URLs

[[206280, 'https://en.wikipedia.org/wiki/Dave_Ragone'],
 [483160, 'https://en.wikipedia.org/wiki/Dan_Campbell'],
 [288004, 'https://en.wikipedia.org/wiki/Aaron_Glenn'],
 [610308, 'https://en.wikipedia.org/wiki/Brian_Johnson'],
 [174517, 'https://en.wikipedia.org/wiki/Luke_Getsy'],
 [995605, 'https://en.wikipedia.org/wiki/Shane_Bowen'],
 [373812, 'https://en.wikipedia.org/wiki/Joe_Brady'],
 [283478, 'https://en.wikipedia.org/wiki/Mick_Lombardi'],
 [370429, 'https://en.wikipedia.org/wiki/Renaldo_Hill'],
 [780072, 'https://en.wikipedia.org/wiki/Scott_Turner'],
 [882423, 'https://en.wikipedia.org/wiki/Brian_VanGorder'],
 [403379, 'https://en.wikipedia.org/wiki/Adam_Stenavich'],
 [425989, 'https://en.wikipedia.org/wiki/Jonathan_Gannon'],
 [958787, 'https://en.wikipedia.org/wiki/Tim_Kelly'],
 [726241, 'https://en.wikipedia.org/wiki/Ronnie_Jones'],
 [285329, 'https://en.wikipedia.org/wiki/Kellen_Moore'],
 [737211, 'https://en.wikipedia.org/wiki/John_Butler'],
 [110664, 'https://en.wikipedia.o

In [733]:
for i in range(len(URLs)):
    URL = URLs[i][1]
    for suffix in suffixes:
        if requests.get(URL+suffix).status_code == 200:
            URLs[i][1] = URL+suffix
            break

URLs

[[206280, 'https://en.wikipedia.org/wiki/Dave_Ragone'],
 [483160, 'https://en.wikipedia.org/wiki/Dan_Campbell'],
 [288004, 'https://en.wikipedia.org/wiki/Aaron_Glenn'],
 [610308,
  'https://en.wikipedia.org/wiki/Brian_Johnson_(American_football_coach)'],
 [174517, 'https://en.wikipedia.org/wiki/Luke_Getsy'],
 [995605, 'https://en.wikipedia.org/wiki/Shane_Bowen_(American_football)'],
 [373812, 'https://en.wikipedia.org/wiki/Joe_Brady_(American_football_coach)'],
 [283478, 'https://en.wikipedia.org/wiki/Mick_Lombardi'],
 [370429, 'https://en.wikipedia.org/wiki/Renaldo_Hill'],
 [780072,
  'https://en.wikipedia.org/wiki/Scott_Turner_(American_football_coach)'],
 [882423, 'https://en.wikipedia.org/wiki/Brian_VanGorder'],
 [403379, 'https://en.wikipedia.org/wiki/Adam_Stenavich'],
 [425989, 'https://en.wikipedia.org/wiki/Jonathan_Gannon'],
 [958787, 'https://en.wikipedia.org/wiki/Tim_Kelly_(American_football)'],
 [726241,
  'https://en.wikipedia.org/wiki/Ronnie_Jones_(American_football_coach)

In [734]:
def get_raw_info(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html')
    infobox = soup.find('table', {'class': 'infobox'})
    if type(infobox) == type(None):
        return
    text = []
    trs = infobox.find_all('tr')
    for tr in trs:
        text.append(tr.text)
    return text

In [735]:
raw_infos = []
for URL in URLs:
    print(URL[1])
    raw_info = get_raw_info(URL[1])
    if raw_info == None:
        continue
    for row in raw_info:
        if 'coach' in row.lower():
            raw_infos.append([URL[0], raw_info])
            break

for raw_info in raw_infos:
    print(raw_info[0])
print(len(raw_infos))

https://en.wikipedia.org/wiki/Dave_Ragone
https://en.wikipedia.org/wiki/Dan_Campbell
https://en.wikipedia.org/wiki/Aaron_Glenn
https://en.wikipedia.org/wiki/Brian_Johnson_(American_football_coach)
https://en.wikipedia.org/wiki/Luke_Getsy
https://en.wikipedia.org/wiki/Shane_Bowen_(American_football)
https://en.wikipedia.org/wiki/Joe_Brady_(American_football_coach)
https://en.wikipedia.org/wiki/Mick_Lombardi
https://en.wikipedia.org/wiki/Renaldo_Hill
https://en.wikipedia.org/wiki/Scott_Turner_(American_football_coach)
https://en.wikipedia.org/wiki/Brian_VanGorder
https://en.wikipedia.org/wiki/Adam_Stenavich
https://en.wikipedia.org/wiki/Jonathan_Gannon
https://en.wikipedia.org/wiki/Tim_Kelly_(American_football)
https://en.wikipedia.org/wiki/Ronnie_Jones_(American_football_coach)
https://en.wikipedia.org/wiki/Kellen_Moore
https://en.wikipedia.org/wiki/John_Butler_(American_football_coach)
https://en.wikipedia.org/wiki/Jerome_Henderson
https://en.wikipedia.org/wiki/Andrew_Weidinger
https:/

In [736]:
birthdates = []
for raw_info in raw_infos:
    for row in raw_info[1]:
        if 'born' in row.lower():
            #print(row)
            long_date = '-'.join(re.findall(r'\d+', row))
            birthdates.append([raw_info[0], '-'.join(long_date.split('-')[:-2][:3])])
for i in range(len(birthdates)):
    birthdate = birthdates[i]
    birthdates[i] = [birthdate[0], ('/'.join(birthdate[1].split('-')[1:]).lstrip('0').lstrip('/').replace('/0','/') + '/' + birthdate[1].split('-')[0]).lstrip('/')]

for birthdate in birthdates:
    print(birthdate)


print(len(birthdates))

[206280, '10/3/1979']
[483160, '4/13/1976']
[288004, '7/16/1972']
[610308, '2/16/1987']
[174517, '2/16/1984']
[995605, '12/9/1986']
[373812, '9/23/1989']
[283478, '10/1/1988']
[370429, '11/12/1978']
[780072, '8/7/1982']
[882423, '4/17/1959']
[403379, '3/11/1983']
[425989, '1/4/1983']
[958787, '8/17/1986']
[726241, '10/17/1955']
[285329, '7/5/1988']
[737211, '4/3/1973']
[110664, '8/8/1969']
[992998, '7/10/1982']
[539629, '4/25/1956']
[375072, '8/29/1976']
[756239, '4/5/1985']
[851071, '5/28/1979']
[666583, '11/16/1962']
[431842, '10/2/1969']
[369945, '1/22/1965']
[993089, '9/26/1956']
[406496, '12/7/1968']
[683172, '3/4/1971']
[341586, '10/17/1982']
[736845, '7/19/1976']
[731193, '10/12/1968']
[811211, '1/23/1985']
[993039, '9/27/1968']
[572746, '2/22/1969']
[390608, '1983']
[301431, '8/12/1975']
[164616, '3/15/1979']
[709399, '10/2/1978']
[526637, '2/20/1961']
[133493, '6/21/1935']
[845235, '1/13/1983']
[700580, '5/5/1987']
[603220, '12/27/1968']
[280078, '3/21/1967']
[203150, '2/10/19

In [737]:
for birthdate in birthdates:
    id_, bd = birthdate[0], birthdate[1]
    data.loc[data.CoachID == id_, 'Date of Birth'] = bd

In [738]:
data[:20]

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",NaN,NaN,10/3/1979,NaN,NaN,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",NaN,NaN,4/13/1976,NaN,NaN,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",NaN,NaN,7/16/1972,NaN,NaN,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",NaN,NaN,2/16/1987,NaN,NaN,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",NaN,NaN,2/16/1984,NaN,NaN,NaN,NaN,NaN,NaN,1
5,995605,"Bowen, Shane",NaN,NaN,12/9/1986,NaN,NaN,NaN,NaN,NaN,NaN,1
6,373812,"Brady, Joe",NaN,NaN,9/23/1989,NaN,NaN,NaN,NaN,NaN,NaN,1
7,283478,"Lombardi, Michael Jr.",NaN,NaN,10/1/1988,NaN,NaN,NaN,NaN,NaN,NaN,1
8,370429,"Hill, Renaldo",NaN,NaN,11/12/1978,NaN,NaN,NaN,NaN,NaN,NaN,1
9,780072,"Turner, Scott",NaN,NaN,8/7/1982,NaN,NaN,NaN,NaN,NaN,NaN,1


In [739]:
coach_infos = []

for raw_info in raw_infos:
    enable = False
    for i in range(len(raw_info[1])):
        row = raw_info[1][i]
        if 'as a coach' in row.lower():
            coach_infos.append([raw_info[0], raw_info[1][i+1].split('\n')])
            break

for coach_info in coach_infos:
    coach_info[1] = [row for row in coach_info[1] if row]
coach_infos[0]

[206280,
 ['Hartford Colonials (2010)Quarterbacks coach',
  'Tennessee Titans (2011–2012)Wide receivers coach',
  'Tennessee Titans (2013)Quarterbacks coach',
  'Washington Redskins (2015)Offensive quality control coach',
  'Chicago Bears (2016–2019)Quarterbacks coach',
  'Chicago Bears (2020)Passing game coordinator',
  'Atlanta Falcons (2021–2023)Offensive coordinator',
  'Los Angeles Rams (2024–present)Quarterbacks coach']]

In [740]:
for coach_info in coach_infos:
    sections_total = []
    for row in coach_info[1]:
        sections = []
        for i in range(len(row)):
            if row[i] == '(' and ord(row[i+1]) >= 48 and ord(row[i+1]) <= 57:
                sections.append(row[:i].strip())
                start_index = i+1
            if row[i] == ')' and ((ord(row[i-1]) >= 48 and ord(row[i-1]) <= 57) or row[i-1] == 't'):
                sections.append(row[start_index:i].strip())
                sections.append(row[i+1:].strip())
        sections_total.append(sections)
    coach_info[1] = sections_total

for coach_info in coach_infos:
    for row in coach_info[1]:
        print(row)
    print()

['Hartford Colonials', '2010', 'Quarterbacks coach']
['Tennessee Titans', '2011–2012', 'Wide receivers coach']
['Tennessee Titans', '2013', 'Quarterbacks coach']
['Washington Redskins', '2015', 'Offensive quality control coach']
['Chicago Bears', '2016–2019', 'Quarterbacks coach']
['Chicago Bears', '2020', 'Passing game coordinator']
['Atlanta Falcons', '2021–2023', 'Offensive coordinator']
['Los Angeles Rams', '2024–present', 'Quarterbacks coach']

['Miami Dolphins', '2010', 'Coaching intern']
['Miami Dolphins', '2011–2015', 'Tight ends coach']
['Miami Dolphins', '2015', 'Interim head coach']
['New Orleans Saints', '2016–2020', 'Assistant head coach & tight ends coach']
['Detroit Lions', '2021–present', 'Head coach']

['Cleveland Browns', '2014–2015', 'Assistant defensive backs coach']
['New Orleans Saints', '2016–2020', 'Defensive backs coach']
['Detroit Lions', '2021–present', 'Defensive coordinator']

['Utah', '2010–2011', 'Quarterbacks coach']
['Utah', '2012–2013', 'Offensive coor

In [741]:
def get_year_list(years):
    if '–' not in years:
        year_list = [int(years)]
        return year_list
    if 'present' in years.lower():
        year_list = list(range(int(years.split('–')[0]), 2025))
    else:
        year_list = list(range(int(years.split('–')[0]), int(years.split('–')[1])+1))

    return year_list


for coach_info in coach_infos:
    for row in coach_info[1]:
        years = row[1].replace('−', '–').replace('-', '–')
        if ',' in years:
            years_sects = years.split(', ')
            year_list = []
            for sect in years_sects:
                year_list = year_list + get_year_list(sect)
        else:
            year_list = get_year_list(years)

        row[1] = year_list

for coach_info in coach_infos:
    for row in coach_info[1]:
        print(row)
    print()

['Hartford Colonials', [2010], 'Quarterbacks coach']
['Tennessee Titans', [2011, 2012], 'Wide receivers coach']
['Tennessee Titans', [2013], 'Quarterbacks coach']
['Washington Redskins', [2015], 'Offensive quality control coach']
['Chicago Bears', [2016, 2017, 2018, 2019], 'Quarterbacks coach']
['Chicago Bears', [2020], 'Passing game coordinator']
['Atlanta Falcons', [2021, 2022, 2023], 'Offensive coordinator']
['Los Angeles Rams', [2024], 'Quarterbacks coach']

['Miami Dolphins', [2010], 'Coaching intern']
['Miami Dolphins', [2011, 2012, 2013, 2014, 2015], 'Tight ends coach']
['Miami Dolphins', [2015], 'Interim head coach']
['New Orleans Saints', [2016, 2017, 2018, 2019, 2020], 'Assistant head coach & tight ends coach']
['Detroit Lions', [2021, 2022, 2023, 2024], 'Head coach']

['Cleveland Browns', [2014, 2015], 'Assistant defensive backs coach']
['New Orleans Saints', [2016, 2017, 2018, 2019, 2020], 'Defensive backs coach']
['Detroit Lions', [2021, 2022, 2023, 2024], 'Defensive coord

In [742]:
nfl_teams = pd.read_csv('nfl_teams.csv')
nfl_teams

,Name,Abbreviation,Conference,Division
0,Arizona Cardinals,ARI,NFC,West
1,Atlanta Falcons,ATL,NFC,South
2,Baltimore Ravens,BAL,AFC,North
3,Buffalo Bills,BUF,AFC,East
4,Carolina Panthers,CAR,NFC,South
5,Chicago Bears,CHI,NFC,North
6,Cincinnati Bengals,CIN,AFC,North
7,Cleveland Browns,CLE,AFC,North
8,Dallas Cowboys,DAL,NFC,East
9,Denver Broncos,DEN,AFC,West


In [743]:
teams = nfl_teams['Name'].values.tolist()
coach_nfl_exps, coach_other_exps = [], []
for coach_info in coach_infos:
    coach_nfl_exp = []
    coach_other_exp = []
    nfl_years = []
    other_years = []
    for row in coach_info[1]:
        for i in range(len(teams)):
            team = teams[i]
            if team in row[0]:
                nfl_years = nfl_years + row[1]
                coach_nfl_exp.append(row)
                break
        
            if i == len(teams)-1:
                other_years = other_years + row[1]
                coach_other_exp.append(row)
    coach_nfl_exps.append([coach_info[0], coach_nfl_exp])
    coach_other_exps.append([coach_info[0], coach_other_exp])
    if 2024 in nfl_years:
        nfl_years.remove(2024)
    if 2024 in other_years:
        other_years.remove(2024)
    coach_info.append(len(list(set(nfl_years))))
    coach_info.append(len(list(set(other_years))))

for coach_info in coach_infos:
    print(coach_info[0], coach_info[-1], coach_info[-2])

206280 1 12
483160 0 14
288004 0 10
610308 11 3
174517 8 9
995605 7 8
373812 5 6
283478 2 11
370429 5 6
780072 7 12
403379 6 7
425989 4 13
958787 6 10
285329 0 6
737211 19 10
110664 0 17
992998 11 12
539629 34 10
375072 8 15
756239 5 9
851071 0 10
666583 23 15
431842 0 17
369945 26 11
993089 12 31
406496 2 9
683172 15 16
341586 4 15
736845 6 11
731193 19 13
811211 6 11
993039 6 11
572746 0 18
390608 12 9
301431 5 14
164616 0 11
709399 8 11
526637 2 13
133493 10 39
845235 6 11
700580 5 3
603220 28 5
203150 8 1
773662 15 1
617503 0 2
584846 0 5
769801 0 3
179035 2 5
679155 6 5
243973 15 15
798757 0 2
978151 8 1
405883 25 5
568069 7 8
683610 21 7
337183 6 15
597471 8 2
326054 2 3
563390 7 1
879655 14 2
602717 0 5
322344 19 1
447099 7 7
453987 10 2
721490 11 2
616531 0 7
559160 10 2
348442 0 1
112170 16 8
575537 0 1
399596 15 9
863935 14 1
597115 1 6
188452 0 4
818574 1 10
248770 39 0
102084 3 3
337463 17 1
685081 3 1
720827 12 1
427570 7 1
729697 8 9
457188 14 8
267557 16 4
365639 10 7
71

In [744]:
for coach_info in coach_infos:
    id_, nfl_yrs, other_yrs = coach_info[0], coach_info[-2], coach_info[-1]
    data.loc[data.CoachID == id_, 'NFL Coach Experience (thru 2023)'] = int(nfl_yrs)
    data.loc[data.CoachID == id_, 'College/Other Exp (thru 2023)'] = int(other_yrs)

In [745]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,NaN,NaN,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,NaN,NaN,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,NaN,NaN,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,NaN,NaN,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",4.0,14.0,8/18/1982,NaN,NaN,NaN,NaN,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",4.0,9.0,10/16/1979,NaN,NaN,NaN,NaN,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [746]:
player_infos = []

for raw_info in raw_infos:
    enable = False
    for i in range(len(raw_info[1])):
        row = raw_info[1][i]
        if 'as a player' in row.lower():
            player_infos.append([raw_info[0], raw_info[1][i+1].split('\n')])
            break
        if i == len(raw_info[1])-1:
            player_infos.append([raw_info[0], ''])

for player_info in player_infos:
    player_info[1] = [row for row in player_info[1] if row]

for player_info in player_infos:
    print(player_info)
    print()

[206280, ['Houston Texans (2003–2005)', 'Berlin Thunder (2005)', 'Cincinnati Bengals (2006)*', 'St. Louis Rams (2006)*', 'Carolina Panthers (2007)*']]

[483160, ['New York Giants (1999–2002)', 'Dallas Cowboys (2003–2005)', 'Detroit Lions (2006–2008)', 'New Orleans Saints (2009)']]

[288004, ['New York Jets (1994–2001)', 'Houston Texans (2002–2004)', 'Dallas Cowboys (2005–2006)', 'Jacksonville Jaguars (2007)', 'New Orleans Saints (2008)']]

[610308, ['New York Sentinels (2009)']]

[174517, ['San Francisco 49ers (2007)*']]

[995605, []]

[373812, []]

[283478, []]

[370429, ['Arizona Cardinals (2001–2004)', 'Oakland Raiders (2005)', 'Miami Dolphins (2006–2008)', 'Denver Broncos (2009–2010)']]

[780072, []]

[882423, []]

[403379, ['Carolina Panthers (2006)*', 'Green Bay Packers (2006–2007)*', '→Amsterdam Admirals (2007)', 'Dallas Cowboys (2008)*', 'Houston Texans (2008–2010)*']]

[425989, []]

[958787, []]

[726241, []]

[285329, ['Detroit Lions (2012–2014)', 'Dallas Cowboys (2015–2017)'

In [747]:
for player_info in player_infos:
    playing_years = []
    for team in player_info[1]:
        team_name, years = team.split(' (')[0], team.split(' (')[1]
        if years[-1] == '*': # We are not accounting for Offseason and practice squad seasons.
            continue
        if team_name not in teams:
            continue
        years = years[:-1].replace('−', '–')
        if '–' in years:
            years = list(range(int(years.split('–')[0]), int(years.split('–')[1])+1))
        else:
            years = [int(years)]
        playing_years = playing_years + years
    player_info.append(len(list(set(playing_years))))

In [748]:
for player_info in player_infos:
    id_, playing_yrs = player_info[0], player_info[-1]
    data.loc[data.CoachID == id_, 'NFL Playing Exp'] = int(playing_yrs)

In [749]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,NaN,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,NaN,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,NaN,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,NaN,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",4.0,14.0,8/18/1982,0.0,NaN,NaN,NaN,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",4.0,9.0,10/16/1979,7.0,NaN,NaN,NaN,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [750]:
for coach_nfl_exp in coach_nfl_exps:
    coach_positions = [exp[-1] for exp in coach_nfl_exp[1]]
    if 'Head coach' in coach_positions or 'Head Coach' in coach_positions:
        coach_nfl_exp.append(1)
        continue
    if 'Offensive coordinator' in coach_positions or 'Defensive coordinator' in coach_positions:
        coach_nfl_exp.append(2)
        continue
    coach_nfl_exp.append(3)

for coach_nfl_exp in coach_nfl_exps:
    print(coach_nfl_exp[0], coach_nfl_exp[-1])

206280 2
483160 1
288004 2
610308 2
174517 2
995605 2
373812 2
283478 2
370429 2
780072 2
403379 2
425989 1
958787 2
285329 2
737211 3
110664 3
992998 3
539629 3
375072 3
756239 3
851071 3
666583 3
431842 3
369945 3
993089 3
406496 3
683172 3
341586 3
736845 3
731193 3
811211 3
993039 3
572746 3
390608 3
301431 3
164616 3
709399 3
526637 3
133493 3
845235 3
700580 3
603220 3
203150 3
773662 3
617503 3
584846 3
769801 3
179035 3
679155 3
243973 3
798757 3
978151 3
405883 3
568069 3
683610 3
337183 3
597471 1
326054 3
563390 3
879655 3
602717 2
322344 3
447099 3
453987 3
721490 3
616531 3
559160 3
348442 3
112170 3
575537 3
399596 3
863935 3
597115 3
188452 3
818574 3
248770 3
102084 3
337463 3
685081 3
720827 3
427570 3
729697 3
457188 3
267557 3
365639 3
718914 3
820092 3
746390 3
120150 3
329662 3
859328 3
303074 3
129324 3
327851 3
557478 3
576164 3
362130 3
434898 3
375521 3
762540 3
148663 3
301943 3
890486 3
185263 3
730299 3
767109 3
151320 2
404778 3
393708 3
405192 3
917339 3
6

In [751]:
for coach_nfl_exp in coach_nfl_exps:
    id_, exp = coach_nfl_exp[0], coach_nfl_exp[-1]
    data.loc[data.CoachID == id_, 'Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)'] = int(exp)

In [752]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,2.0,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,1.0,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,2.0,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,2.0,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,2.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",4.0,14.0,8/18/1982,0.0,3.0,NaN,NaN,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",4.0,9.0,10/16/1979,7.0,3.0,NaN,NaN,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [753]:
for coach_nfl_exp in coach_nfl_exps:
    coord_years = []
    for row in coach_nfl_exp[1]:
        if 'Offensive coordinator' in row[-1] or 'Defensive coordinator' in row[-1]:
            coord_years = coord_years + row[1]
    if 2024 in coord_years:
        coord_years.remove(2024)
    coach_nfl_exp.append(len(list(set(coord_years))))

In [754]:
for coach_nfl_exp in coach_nfl_exps:
    id_, coord_yrs = coach_nfl_exp[0], coach_nfl_exp[-1]
    data.loc[data.CoachID == id_, '# seasons as NFL Coordinator'] = int(coord_yrs)

In [755]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,2.0,NaN,3.0,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,1.0,NaN,0.0,NaN,NaN,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,2.0,NaN,3.0,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,2.0,NaN,1.0,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,2.0,NaN,2.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",4.0,14.0,8/18/1982,0.0,3.0,NaN,0.0,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",4.0,9.0,10/16/1979,7.0,3.0,NaN,0.0,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [756]:
for coach_nfl_exp in coach_nfl_exps:
    id_ = coach_nfl_exp[0]
    first_year_hc = np.nan
    for row in coach_nfl_exp[1]:
        if 'Head coach' == row[-1] or 'head coach' == row[-1] or 'Interim head coach' == row[-1]:
            first_year_hc = int(row[1][0])
            break
    data.loc[data.CoachID == id_, '1st Season as NFL HC'] = first_year_hc

In [757]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,2.0,NaN,3.0,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,1.0,NaN,0.0,NaN,2015.0,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,2.0,NaN,3.0,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,2.0,NaN,1.0,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,2.0,NaN,2.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",4.0,14.0,8/18/1982,0.0,3.0,NaN,0.0,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",4.0,9.0,10/16/1979,7.0,3.0,NaN,0.0,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [758]:
for coach_other_exp in coach_other_exps:
    for row in coach_other_exp[1]:
        print(row)
    print()

['Hartford Colonials', [2010], 'Quarterbacks coach']



['Utah', [2010, 2011], 'Quarterbacks coach']
['Utah', [2012, 2013], 'Offensive coordinator & quarterbacks coach']
['Mississippi State', [2014, 2015, 2016], 'Quarterbacks coach']
['Houston', [2017], 'Offensive coordinator & quarterbacks coach']
['Florida', [2018, 2019], 'Quarterbacks coach']
['Florida', [2020], 'Offensive coordinator & quarterbacks coach']

['Akron', [2007, 2008], 'Graduate assistant']
['West Virginia Wesleyan', [2009], 'Offensive coordinator & quarterbacks coach']
['Pittsburgh', [2010], 'Graduate assistant']
['Indiana (PA)', [2011, 2012], 'Offensive coordinator & quarterbacks coach']
['Western Michigan', [2013], 'Wide receivers coach']
['Mississippi State', [2018], 'Offensive coordinator & wide receivers coach']

['Georgia Tech', [2009], 'Student assistant']
['Georgia Tech', [2010, 2011], 'Graduate assistant']
['Ohio State', [2012], 'Graduate assistant']
['Kennesaw State', [2013, 2014, 2015], 'Linebackers coach']


In [759]:
pos_dict = {'head coach': ['HC','Off',1],
            'quarterbacks': ['QB','Off',3],
            'offensive coordinator': ['OC','Off',2],
            'defensive coordinator': ['DC','Def',2],
            'graduate assistant': ['GA','X',4],
            'wide receivers': ['WR','Off',3],
            'student assistant': ['GA','X',4],
            'linebackers': ['LB','Def',3],
            'offensive line': ['OL','Off',3],
            'defensive line': ['DL','Dff',3],
            'special teams': ['ST','ST',3],
            'special teams coordinator': ['ST','ST',2],
            'defensive backs': ['DB','Def',3],
            'safeties': ['DB','Def',3],
            'running backs': ['RB','Off',3],
            'tight ends': ['TE','Off',3],
            'cornerbacks': ['DB','Def',3],
            'assistant head coach': ['AH','X',3],
            'assistant ': ['AP','X',3]}

In [760]:
for coach_other_exp in coach_other_exps:
    pos_history = []
    for row in coach_other_exp[1]:
        position = row[-1].lower()
        for pos in pos_dict:
            if pos in position:
                pos_history.append(pos_dict[pos])

    if len(pos_history) == 0:
        continue
    highest_pos = pos_history[-1]
    for pos in pos_history[::-1]:
        if pos[-1] < highest_pos[-1]:
            highest_pos = pos

    id_ = coach_other_exp[0]
    data.loc[data.CoachID == id_, 'Highest Pos College / other Pro'] = highest_pos[0]

In [761]:
data[:20]

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,2.0,QB,3.0,NaN,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,1.0,NaN,0.0,NaN,2015.0,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,2.0,NaN,3.0,NaN,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,2.0,OC,1.0,NaN,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,2.0,OC,2.0,NaN,NaN,1
5,995605,"Bowen, Shane",8.0,7.0,12/9/1986,0.0,2.0,LB,3.0,NaN,NaN,1
6,373812,"Brady, Joe",6.0,5.0,9/23/1989,0.0,2.0,WR,2.0,NaN,NaN,1
7,283478,"Lombardi, Michael Jr.",11.0,2.0,10/1/1988,0.0,2.0,NaN,2.0,NaN,NaN,1
8,370429,"Hill, Renaldo",6.0,5.0,11/12/1978,10.0,2.0,DB,2.0,NaN,NaN,1
9,780072,"Turner, Scott",12.0,7.0,8/7/1982,0.0,2.0,OC,3.0,NaN,NaN,1


In [762]:
for coach_info in coach_infos:
    sides = []
    #print(coach_info[0])
    for row in coach_info[1]:
        position = row[-1].lower()
        if 'offensive' in position:
            sides.append('Off')
            continue
        if 'defensive' in position:
            sides.append('Def')
            continue
        for pos in pos_dict:
            if pos in position:
                sides.append(pos_dict[pos][1])
                break
    if len(sides) == 0:
        coach_info.append(np.nan)
    else:
        coach_info.append(max(set(sides), key = sides.count))

In [763]:
for coach_info in coach_infos:
    id_, side = coach_info[0], coach_info[-1]
    data.loc[data.CoachID == id_, 'Off/Def side of ball'] = side

In [764]:
data[:20]

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",12.0,1.0,10/3/1979,3.0,2.0,QB,3.0,Off,NaN,1
1,483160,"Campbell, Dan",14.0,0.0,4/13/1976,11.0,1.0,NaN,0.0,Off,2015.0,1
2,288004,"Glenn, Aaron",10.0,0.0,7/16/1972,15.0,2.0,NaN,3.0,Def,NaN,1
3,610308,"Johnson, Brian",3.0,11.0,2/16/1987,0.0,2.0,OC,1.0,Off,NaN,1
4,174517,"Getsy, Luke",9.0,8.0,2/16/1984,0.0,2.0,OC,2.0,Off,NaN,1
5,995605,"Bowen, Shane",8.0,7.0,12/9/1986,0.0,2.0,LB,3.0,Def,NaN,1
6,373812,"Brady, Joe",6.0,5.0,9/23/1989,0.0,2.0,WR,2.0,Off,NaN,1
7,283478,"Lombardi, Michael Jr.",11.0,2.0,10/1/1988,0.0,2.0,NaN,2.0,Off,NaN,1
8,370429,"Hill, Renaldo",6.0,5.0,11/12/1978,10.0,2.0,DB,2.0,Def,NaN,1
9,780072,"Turner, Scott",12.0,7.0,8/7/1982,0.0,2.0,OC,3.0,Off,NaN,1
